In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import os
import urllib.parse
import time

# 1. Specify the folder path containing TXT files
folder_path = r"C:\Users\Advice\Desktop\HTMLextract\scrape_results"

# 2. Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

# 3. Initialize the embeddings
os.environ["AZURE_OPENAI_API_KEY"] = 'd8e93330e8384f06aa1c8ace726af49e'
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dataiku-gpt4ommi.openai.azure.com'
from langchain_community.embeddings.azure_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="gpt-4o",
    model="text-embedding-3-large",
    openai_api_type="azure",
    openai_api_base="https://dataiku-gpt4ommi.openai.azure.com",
    openai_api_version = "2023-05-15"
)

# Function to process documents in batches
def process_documents_in_batches(documents, batch_size=1):
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        yield FAISS.from_documents(batch, embeddings)
        time.sleep(1)  # 1 second delay between batches

# 4. Process all TXT files in the folder
all_texts = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):
        try:
            decoded_file_name = urllib.parse.unquote(file_name)
            file_path = os.path.join(folder_path, decoded_file_name)
            short_path = os.path.abspath(file_path)
            loader = TextLoader(short_path, encoding='utf-8')
            documents = loader.load()
            texts = text_splitter.split_documents(documents)
            all_texts.extend(texts)
            print(f"Processed file: {decoded_file_name}")
            time.sleep(5)  # 5 second delay between files
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")

# 5. Create combined vectorstore with delay between embeddings
combined_vectorstore = FAISS.from_documents(all_texts[:1], embeddings)  # Initialize with first document
for vs in process_documents_in_batches(all_texts[1:], batch_size=1):
    combined_vectorstore.merge_from(vs)

# 6. Save the vectorstore
combined_vectorstore.save_local("combined_vectorstore")

c:\Users\Advice\Desktop\PTTC\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


Processed file: 1.txt
Processed file: 10.txt
Processed file: 11.txt
Processed file: 12.txt
Processed file: 13.txt
Processed file: 14.txt
Processed file: 15.txt
Processed file: 16.txt
Processed file: 17.txt
Processed file: 18.txt
Processed file: 19.txt
Processed file: 2.txt
Processed file: 20.txt
Processed file: 21.txt
Processed file: 3.txt
Processed file: 4.txt
Processed file: 5.txt
Processed file: 6.txt
Processed file: 7.txt
Processed file: 8.txt
Processed file: 9.txt
Processed file: webpage_structure__category_career_.txt
Processed file: webpage_structure__mobile-app-presscon_.txt
Processed file: webpage_structure__personal-accident-insurance-tpb_.txt
Processed file: webpage_structure__privacy-notice_.txt
Processed file: webpage_structure__responsible-lending-announce_.txt
Processed file: webpage_structure__responsible-lending_announce_.txt
Processed file: webpage_structure__sawad-excellent-cg-scoring-2024_.txt
Processed file: webpage_structure__srisawad-app_.txt
Processed file: webp

In [5]:
import pickle
embeddings_file_path = "embeddings.pkl"
with open(embeddings_file_path, "wb") as f:
    pickle.dump(embeddings, f)

print(f"Embeddings saved to {embeddings_file_path}")

Embeddings saved to embeddings.pkl


In [1]:
import pickle
from langchain.vectorstores import FAISS
from langchain.tools import StructuredTool
from langchain.chains import RetrievalQA, ConversationChain
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from typing import Dict
import os
# 1. Load the saved embeddings
os.environ["AZURE_OPENAI_API_KEY"] = 'd8e93330e8384f06aa1c8ace726af49e'
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dataiku-gpt4ommi.openai.azure.com'
from langchain_community.embeddings.azure_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="gpt-4o",
    model="text-embedding-3-large",
    openai_api_type="azure",
    openai_api_base="https://dataiku-gpt4ommi.openai.azure.com",
    openai_api_version = "2023-05-15"
)
with open("embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)

# 2. Load the saved vectorstore with allow_dangerous_deserialization set to True
combined_vectorstore = FAISS.load_local("combined_vectorstore", embeddings, allow_dangerous_deserialization=True)

# 3. Create a combined retriever
combined_retriever = combined_vectorstore.as_retriever(search_kwargs={"k": 5})

# 4. Initialize the language model
llm = AzureChatOpenAI(
    openai_api_version="2024-08-01-preview",
    openai_api_base="https://dataiku-gpt4ommi.openai.azure.com",
    azure_deployment="dataiku-ssci-gpt-4o", 
)

# 5. Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=combined_retriever,
    return_source_documents=True
)

# 6. Create a dictionary to store user memories
user_memories = {}

pre_history = [
    {"input": "system", "output": "คุณคือ พนักงานของศรีสวัสดิ์ซึ่งเป็นบริษัทปล่อยสินเชื่อ"},
    {"input": "assistant", "output": "เป้าหมายของคุณคือตอบคำถามเกี่ยวกับบริษัทนี้เท่านั้นห้ามพูดถึงเรื่องอื่นๆ"},
    {"input": "assistant", "output": "โปรดตอบคำถามให้เป็นภาษาเดียวกับที่ถูกถามมา"},
    {"input": "assistant", "output": "โปรดอบคำถามอย่างละเอียดเสมอๆ"}
]


def add_pre_history(user_id, pre_history):
    """Add pre-existing history to the conversation for a specific user."""
    if user_id not in user_memories:
        user_memories[user_id] = ConversationBufferMemory()
    for entry in pre_history:
        user_memories[user_id].save_context({"input": entry['input']}, {"output": entry['output']})


from typing import Dict, List

def process_query(query: str, user_id: str) -> Dict[str, any]:
    # # Get or create memory for the user
    # if user_id not in user_memories:
    #     add_pre_history(user_id, pre_history)

    # Create a conversation chain with user-specific memory
    conversation = ConversationChain(
        llm=llm,
        memory=user_memories[user_id],
        verbose=True
    )

    # Use the QA chain to get the initial answer
    qa_result = qa_chain({"query": query})
    initial_answer = qa_result['result']
    sources = qa_result['source_documents']

    # Use the conversation chain to generate a response that includes memory
    conversation_result = conversation.predict(input=f"Question: {query}\nInitial Answer: {initial_answer}")

    # Prepare the structured sources
    structured_sources = []
    for i, doc in enumerate(sources, 1):
        structured_sources.append({
            "number": i,
            "source": doc.metadata.get('source', 'Unknown'),
            "excerpt": doc.page_content[:200] + "..."
        })

    # Prepare the structured response
    structured_response = {
        "query": query,
        "answer": conversation_result,
        "initial_answer": initial_answer,
        "sources": structured_sources
    }

    return structured_response

# Update the StructuredTool to reflect the new return type
kpp_document_tool = StructuredTool.from_function(
    name="เอกสารการปล่อยสินเชื่อต่างๆ",
    func=process_query,
    description="นี่คือข้อมูลและเอกสารการปล่อยสินเชื่อในบริษัทศรีสวัสดิ์ซึ่งมีข้อมูลครบถ้วนทั้งหมด",
    return_direct=True
)

c:\Users\Advice\Desktop\PTTC\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(
c:\Users\Advice\Desktop\PTTC\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [2]:
import queue
import threading
from datetime import datetime
from pyngrok import ngrok, conf
from flask import Flask, request, abort
from langchain.chains import ConversationChain
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
from pyngrok import ngrok
import requests
app = Flask(__name__)
line_bot_api = LineBotApi('DvWUVF/W/a7xm7fwBNKpq6rUX6WsEs1YdZPYyXh4wg0xwpSnSloEtBO5ftkO9JDgSBHIVUjqbxJ97OH9dqq/fB3YzSyR0hjeQxz4cr/lPSUq1oy8dX7NKQ1/56B0+Ur2FQwSWZ9RqU0cum8p6Sk0KAdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('8e9ff9ff62f7db3d6bb3e3966ab1d740')

# Dictionary to hold queues for each user
user_queues = {}
# Lock to manage sequential task dependencies
sequential_task_lock = threading.Lock()
def user_worker(user_id):
    while True:
        task = user_queues[user_id].get()
        if task is None:
            break  # Exit condition for the worker thread
        data, file_name, target_function = task
        target_function(data, file_name)
        user_queues[user_id].task_done()

# Function to add tasks to the user's queue
def add_task(user_id, data, file_name, target_function):
    if user_id not in user_queues:
        user_queues[user_id] = queue.Queue()
        threading.Thread(target=user_worker, args=(user_id,)).start()
    user_queues[user_id].put((data, file_name, target_function))

# Function to check remaining tasks for a specific user
def check_remaining_tasks(user_id):
    if user_id in user_queues:
        return user_queues[user_id].qsize()
    else:
        return 0

In [3]:
def chat_with_history(data, id):
    user_input = data['events'][0]['message']['text']
    user_id = data['events'][0]['source']['userId']
    """Interact with the chatbot and maintain conversation history for each user."""
    if user_id not in user_memories:
        add_pre_history(user_id, pre_history)
    response = kpp_document_tool.run({"query": user_input, "user_id": user_id})
    if check_remaining_tasks(user_id) == 0:
        line_bot_api.reply_message(data['events'][0]['replyToken'], TextSendMessage(text=str(response['answer'])))

In [ ]:
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_text_message():
    data = request.get_json()
    user_id = data['events'][0]['source']['userId']
    add_task(user_id, data, 'test', chat_with_history)
    return 'OK', 200

if __name__ == '__main__':
    authtoken = "2MjXYYm8LecquhskfM41kZc8i62_Jrf3esZ4DsA2QYWYqQAF"
    conf.get_default().auth_token = authtoken
    CHANNEL_ACCESS_TOKEN = 'DvWUVF/W/a7xm7fwBNKpq6rUX6WsEs1YdZPYyXh4wg0xwpSnSloEtBO5ftkO9JDgSBHIVUjqbxJ97OH9dqq/fB3YzSyR0hjeQxz4cr/lPSUq1oy8dX7NKQ1/56B0+Ur2FQwSWZ9RqU0cum8p6Sk0KAdB04t89/1O/w1cDnyilFU='

    # Start the initial ngrok tunnel
    url = ngrok.connect(2000)
    print(f"ngrok tunnel opened at {url.public_url}")
    new_webhook_url = url.public_url+'/callback'

    # Set the headers
    headers = {
        'Authorization': f'Bearer {CHANNEL_ACCESS_TOKEN}',
        'Content-Type': 'application/json'
    }

    # Set the data payload
    data = {
        'endpoint': new_webhook_url
    }

    # Make the PUT request
    response = requests.put('https://api.line.me/v2/bot/channel/webhook/endpoint', headers=headers, json=data)

    # Check the response
    if response.status_code == 200:
        print('Webhook URL updated successfully')
    else:
        print('Failed to update Webhook URL')
        print('Response:', response.json())
    app.run(host='0.0.0.0', port=2000)

ngrok tunnel opened at https://c43c-180-180-140-150.ngrok-free.app
Webhook URL updated successfully
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:2000
 * Running on http://192.168.1.2:2000
Press CTRL+C to quit
127.0.0.1 - - [12/Dec/2024 09:04:30] "POST /callback HTTP/1.1" 200 -
c:\Users\Advice\Desktop\PTTC\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: system
AI: คุณคือ พนักงานของศรีสวัสดิ์ซึ่งเป็นบริษัทปล่อยสินเชื่อ
Human: assistant
AI: เป้าหมายของคุณคือตอบคำถามเกี่ยวกับบริษัทนี้เท่านั้นห้ามพูดถึงเรื่องอื่นๆ
Human: assistant
AI: โปรดตอบคำถามให้เป็นภาษาเดียวกับที่ถูกถามมา
Human: assistant
AI: โปรดอบคำถามอย่างละเอียดเสมอๆ
Human: Question: สินเชื่อรถ
Initial Answer: คุณสามารถขอสินเชื่อรถได้ที่ "ศรีสวัสดิ์ เงินสดทันใจ" โดยมีรายละเอียดสินเชื่อสำหรับรถยนต์และรถมอเตอร์ไซค์ดังนี้:

**สินเชื่อรถยนต์:**
- รับวงเงินสูง
- ไม่ต้องโอนเล่มทะเบียน
- ไม่ต้องใช้คนค้ำประกัน
- รับรีไฟแนนซ์รถยนต์
- อัตราดอกเบี้ยที่แท้จริง 19.66% – 24% ต่อปี
- มีสาขาพร้อมให้บริการกว่า 5,500 สาขา

**เอกสารเบื้องต้นในการสมัครสินเชื่อรถยนต์:**
- บัตร

127.0.0.1 - - [12/Dec/2024 09:23:30] "POST /callback HTTP/1.1" 200 -




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: system
AI: คุณคือ พนักงานของศรีสวัสดิ์ซึ่งเป็นบริษัทปล่อยสินเชื่อ
Human: assistant
AI: เป้าหมายของคุณคือตอบคำถามเกี่ยวกับบริษัทนี้เท่านั้นห้ามพูดถึงเรื่องอื่นๆ
Human: assistant
AI: โปรดตอบคำถามให้เป็นภาษาเดียวกับที่ถูกถามมา
Human: assistant
AI: โปรดอบคำถามอย่างละเอียดเสมอๆ
Human: Question: I wanna apply for a car credit
Initial Answer: To apply for a car credit with Srisawad (ศรีสวัสดิ์ เงินสดทันใจ), you need to provide the following documents:

1. Identification card
2. House registration
3. Original car registration book (must be free of obligations)
4. Bank account book
5. Income proof (if available)
6. Copy of the hire-purchase agreement from the previous fi

127.0.0.1 - - [12/Dec/2024 09:24:18] "POST /callback HTTP/1.1" 200 -




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: system
AI: คุณคือ พนักงานของศรีสวัสดิ์ซึ่งเป็นบริษัทปล่อยสินเชื่อ
Human: assistant
AI: เป้าหมายของคุณคือตอบคำถามเกี่ยวกับบริษัทนี้เท่านั้นห้ามพูดถึงเรื่องอื่นๆ
Human: assistant
AI: โปรดตอบคำถามให้เป็นภาษาเดียวกับที่ถูกถามมา
Human: assistant
AI: โปรดอบคำถามอย่างละเอียดเสมอๆ
Human: Question: I wanna apply for a car credit
Initial Answer: To apply for a car credit with Srisawad (ศรีสวัสดิ์ เงินสดทันใจ), you need to provide the following documents:

1. Identification card
2. House registration
3. Original car registration book (must be free of obligations)
4. Bank account book
5. Income proof (if available)
6. Copy of the hire-purchase agreement from the previous fi

127.0.0.1 - - [12/Dec/2024 09:24:54] "POST /callback HTTP/1.1" 200 -




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: system
AI: คุณคือ พนักงานของศรีสวัสดิ์ซึ่งเป็นบริษัทปล่อยสินเชื่อ
Human: assistant
AI: เป้าหมายของคุณคือตอบคำถามเกี่ยวกับบริษัทนี้เท่านั้นห้ามพูดถึงเรื่องอื่นๆ
Human: assistant
AI: โปรดตอบคำถามให้เป็นภาษาเดียวกับที่ถูกถามมา
Human: assistant
AI: โปรดอบคำถามอย่างละเอียดเสมอๆ
Human: Question: I wanna apply for a car credit
Initial Answer: To apply for a car credit with Srisawad (ศรีสวัสดิ์ เงินสดทันใจ), you need to provide the following documents:

1. Identification card
2. House registration
3. Original car registration book (must be free of obligations)
4. Bank account book
5. Income proof (if available)
6. Copy of the hire-purchase agreement from the previous fi